# CRDS reference mappings

This notebook explores the CRDS .rmap rules files.

## Prerequisites

To follow along with the examples in this notebook, you will need:

- The following Python packages installed:

```
$ pip install crds==7.6.1
```

In [ ]:
import crds

## .rmap olympics

Congratulations, you've been chosen to represent your scrum team in the .rmap olympics!  For each of the following challenges, edit the `RMAP` variable according to the instructions.  Test your changes by executing the "Test your solution" code cell.

### Challenge 1

To celebrate your humble notebook author's birthday, we're going to change the NIRCam `photom` reference file to a special edition just for that day.  Modify the following .rmap to select `jwst_nircam_photom_birthday.fits` for the duration of 11/20/2020, but only for the `NRC_IMAGE` exposure type.

In [ ]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nircam_photom_0009.rmap',
    'filekind' : 'PHOTOM',
    'instrument' : 'NIRCAM',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nircam_photom_0010.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : '32715a282c1417c74b06eaac369d1c1bb99ba248',
}

selector = Match({
    ('NRCA1', 'N/A') : UseAfter({
        '1900-01-01 00:00:00' : 'jwst_nircam_photom_0031.fits',
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0048.fits',
    }),
    ('NRCA1', 'NRC_CORON|NRC_FLAT|NRC_FOCUS|NRC_IMAGE|NRC_TACONFIRM|NRC_TACQ|NRC_TSIMAGE') : UseAfter({
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0074.fits',
    }),
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(exposure_type, date, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "NRCA1",
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.TYPE={exposure_type}, META.OBSERVATION.DATE={date}.  Expected {filename}, got {result}"
        raise AssertionError(message)
    
assert_best_ref("SOME EXPOSURE TYPE", "1980-01-01", "jwst_nircam_photom_0031.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2014-01-01", "jwst_nircam_photom_0048.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2020-11-20", "jwst_nircam_photom_0048.fits")


for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_FOCUS", "NRC_IMAGE", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2014-01-01", "jwst_nircam_photom_0074.fits")
    assert_best_ref(exposure_type, "2020-11-21", "jwst_nircam_photom_0074.fits")
    
for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_IMAGE", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2020-11-20", "jwst_nircam_photom_0074.fits")

assert_best_ref("NRC_IMAGE", "2020-11-20", "jwst_nircam_photom_birthday.fits")

print("Success!")

### Challenge 2

Oh no, MIRI was struck by rogue fireworks and the previously stable flat field correction completely changed!  Add time-dependence to the following .rmap.  For each match case, use the existing reference file up to 10pm UTC on 7/4/2024, but switch to the following files (identified by band) at that time:

`LONG`: jwst_miri_flat_0600.fits

`MEDIUM`: jwst_miri_flat_0601.fits

`SHORT`: jwst_miri_flat_0602.fits

In [ ]:
RMAP = """
header = {
    'classes' : ('Match',),
    'derived_from' : 'jwst_miri_flat_0045.rmap',
    'filekind' : 'FLAT',
    'instrument' : 'MIRI',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_miri_flat_0046.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.INSTRUMENT.FILTER', 'META.INSTRUMENT.BAND', 'META.EXPOSURE.READPATT', 'META.SUBARRAY.NAME'),),
    'sha1sum' : '1b42da81d62fb32d927911f3dcae05a980bcf939',
}

selector = Match({
    ('MIRIFULONG', 'N/A', 'LONG', 'N/A', 'FULL') : 'jwst_miri_flat_0541.fits',
    ('MIRIFULONG', 'N/A', 'MEDIUM', 'N/A', 'FULL') : 'jwst_miri_flat_0539.fits',
    ('MIRIFULONG', 'N/A', 'SHORT', 'N/A', 'FULL') : 'jwst_miri_flat_0542.fits',
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(band, date, time, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
            "META.INSTRUMENT.BAND": band,
            "META.SUBARRAY.NAME": "FULL",
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": time,
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.BAND={band}, META.OBSERVATION.DATE={date}, META.OBSERVATION.TIME={time}.  Expected {filename}, got {result}"
        raise AssertionError(message)
        
assert_best_ref("LONG", "2023-01-01", "00:00:00", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "09:59:59", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "10:00:00", "jwst_miri_flat_0600.fits")
assert_best_ref("LONG", "2025-01-01", "00:00:00", "jwst_miri_flat_0600.fits")

assert_best_ref("MEDIUM", "2023-01-01", "00:00:00", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "09:59:59", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "10:00:00", "jwst_miri_flat_0601.fits")
assert_best_ref("MEDIUM", "2025-01-01", "00:00:00", "jwst_miri_flat_0601.fits")

assert_best_ref("SHORT", "2023-01-01", "00:00:00", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "09:59:59", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "10:00:00", "jwst_miri_flat_0602.fits")
assert_best_ref("SHORT", "2025-01-01", "00:00:00", "jwst_miri_flat_0602.fits")

print("Success!")

### Challenge 3

Construct a selector for the `nirspec` `disperser` reference type with the following rules:

- If the exposure type is `NRS_DARK`, return reference file not applicable.  This rule overrides any subsequent rule.
- If the grating is `G140H` and the exposure type one of (`NRS_AUTOWAVE`, `NRS_AUTOFLAT`, or `NRS_TACQ`), return `jwst_nirspec_disperser_0001.asdf`.
- If the grating is `G140H` and the exposure type not already covered by a previous rule, return `jwst_nirspec_disperser_0002.asdf`.
- If the grating is `MIRROR`, return `jwst_nirspec_disperser_0003.asdf` regardless of exposure type (except `NRS_DARK`).

The selector should be time-dependent, but this initial set of files can all share a useafter timestamp of `2020-01-01 00:00:00`.

In [ ]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nirspec_disperser_0019.rmap',
    'filekind' : 'DISPERSER',
    'instrument' : 'NIRSPEC',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nirspec_disperser_0019.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.GRATING', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : 'ad74383edba73deabd88922565b5f8da7237d779',
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(grating, exposure_type, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.GRATING": grating,
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": "2020-01-01",
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.INSTRUMENT.GRATING={grating}, META.EXPOSURE.TYPE={exposure_type}.  Expected {filename}, got {result}"
        raise AssertionError(message)
        
for grating in ["G140H", "MIRROR", "PRISM"]:
    assert_best_ref(grating, "NRS_DARK", "NOT FOUND n/a")
    
for exposure_type in ["NRS_AUTOWAVE", "NRS_AUTOFLAT", "NRS_TACQ"]:
    assert_best_ref("G140H", exposure_type, "jwst_nirspec_disperser_0001.asdf")
    
import random
import string
random_exposure_type = "NRS_" + "".join([random.choice(string.ascii_uppercase) for _ in range(8)])

assert_best_ref("G140H", random_exposure_type, "jwst_nirspec_disperser_0002.asdf") 

for exposure_type in ["NRS_AUTOWAVE", "NRS_AUTOFLAT", "NRS_TACQ", random_exposure_type]:
    assert_best_ref("MIRROR", exposure_type, "jwst_nirspec_disperser_0003.asdf")
    assert_best_ref("PRISM", exposure_type, "NOT FOUND No match found.")
    
print("Success!")

### Challenge 4

Construct a selector for the `nirspec` `dflat` reference type with the following rules:

- For the `NRS1` detector and any exposure type that begins with `NRS_` select the `jwst_nirspec_dflat_0001.fits` reference file.  Other exposure types should select `jwst_nirspec_dflat_0002.fits`.
- For the `NRS2` detector and the following exposure types: `NRS_MSATA`, `NRS_WATA`, `NRS_CONFIRM`, select `jwst_nirspec_dflat_0003.fits`.  For `NRS_FOCUS`, select `jwst_nirspec_dflat_0004.fits`.  Other exposure types should select `jwst_nirspec_dflat_0005.fits`.

The selector should be time-dependent, but this initial set of files can all share a useafter timestamp of `2020-01-01 00:00:00`.

Hint: you will need to use a `Match` feature that hasn't been discussed in this notebook.  Consult the [CRDS User Manual](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html) for assistance.

In [1]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nirspec_dflat_0005.rmap',
    'filekind' : 'DFLAT',
    'instrument' : 'NIRSPEC',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nirspec_dflat_0005.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : '3ee07dcd1fe41ad6d5e4e1b4dbcf3eaa4369659e',
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(detector, exposure_type, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": detector,
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": "2020-01-01",
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.INSTRUMENT.DETECTOR={detector}, META.EXPOSURE.TYPE={exposure_type}.  Expected {filename}, got {result}"
        raise AssertionError(message)

import random
import string
random_nrs_exposure_type = "NRS_" + "".join([random.choice(string.ascii_uppercase) for _ in range(random.randint(4, 30))])
random_other_exposure_type = "".join([random.choice(string.ascii_uppercase) for _ in range(random.randint(4, 30))])
        
for exposure_type in ["NRS_MSATA", "NRS_WATA", "NRS_CONFIRM", random_nrs_exposure_type]:
    assert_best_ref("NRS1", exposure_type, "jwst_nirspec_dflat_0001.fits")
    
assert_best_ref("NRS1", random_other_exposure_type, "jwst_nirspec_dflat_0002.fits")
    
for exposure_type in ["NRS_MSATA", "NRS_WATA", "NRS_CONFIRM"]:
    assert_best_ref("NRS2", exposure_type, "jwst_nirspec_dflat_0003.fits")

assert_best_ref("NRS2", "NRS_FOCUS", "jwst_nirspec_dflat_0004.fits")

for exposure_type in [random_nrs_exposure_type, random_other_exposure_type]:
    assert_best_ref("NRS2", exposure_type, "jwst_nirspec_dflat_0004.fits") 
    
print("Success!")

### Challenge 5

You're throwing a series of dinner parties!  You've stored your favorite recipes in individual ASDF files and need an .rmap that will select them based on metadata from your guest lists.  A guest list has the following metadata available:

**META.PARTY.GUEST_COUNT**: A positive integer.

**META.PARTY.DIET**: One of `VEGAN`, `VEGETARIAN`, or `OMNIVORE`.

**META.PARTY.ALLERGEN**: A string identifier for a single allergen.  The full set of allergens is unknown, but you're confident that your recipe metadata is complete.

**META.PARTY.DATE**: The date of the party.

**META.PARTY.TIME**: The time of the party.

Here are your recipes:

**mac_and_cheese.asdf**
```
Servings: 100
Allergens: WHEAT, DAIRY
Incompatible diets: VEGAN
Seasonal availability: (unrestricted)
```

**mac_and_cheese_with_asparagus.asdf**
```
Servings: 100
Allergens: `WHEAT`, `DAIRY`
Incompatible diets: `VEGAN`
Seasonal availability: February 1st, 2022 until June 1st, 2022
```

**stir_fry_with_beef.asdf**
```
Servings: 4
Allergens: (none)
Incompatible diets: `VEGAN`, `VEGETARIAN`
Seasonal availability: (unrestricted)
```

**oatmeal.asdf**
```
Servings: (unlimited)
Allergens: (none)
Incompatible diets: (none)
Seasonal availability: (unrestricted)
```

**oatmeal_with_apples.asdf**
```
Servings: (unlimited)
Allergens: (none)
Incompatible diets: (none)
Seasonal availability: August 1st, 2022 until November 1st, 2022
```

Serving oatmeal should be avoided whenever possible.  Prioritize seasonal vegetables.  In anticipation of continued SARS-CoV-2 restrictions, the parties are planned for the year 2022.

In [ ]:
RMAP = """
header = {
    # ???
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(guest_count, diet, allergen, date):
    result = rmap.get_best_ref(
        {
            "META.PARTY.GUEST_COUNT": guest_count,
            "META.PARTY.DIET": diet,
            "META.PARTY.ALLERGEN": allergen,
            "META.PARTY.DATE": date,
            "META.PARTY.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.PARTY.GUEST_COUNT={guest_count}, META.PARTY.DIET={diet}, META.PARTY.ALLERGEN={allergen}, META.PARTY.DATE={date}.  Expected {filename}, got {result}"
        raise AssertionError(message)
               
for guest_count in [101, 1000, 10000]:
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-07-01", "oatmeal.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-08-15", "oatmeal_with_apples.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-11-20", "oatmeal.asdf")
    
for guest_count in range(5, 100):
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-01-05", "mac_and_cheese.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-03-08", "mac_and_cheese_with_asparagus")
    assert_best_ref(guest_count, "VEGAN", "N/A", "2022-09-20", "oatmeal.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "WHEAT", "2022-09-20", "oatmeal.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "DAIRY", "2022-09-20", "oatmeal.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "KANGAROO", "2022-01-05", "mac_and_cheese.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "KANGAROO", "2022-03-08", "mac_and_cheese.asdf")

for guest_count in range(1, 5):
    assert_best_ref(guest_count, "OMNIVORE", "WHEAT", "2022-05-01", "stir_fry_with_beef.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "DAIRY", "2022-05-01", "stir_fry_with_beef.asdf")
    assert_best_ref(guest_count, "VEGETARIAN", "WHEAT", "2022-05-01", "oatmeal.asdf")
    assert_best_ref(guest_count, "VEGAN", "WHEAT", "2022-05-01", "oatmeal.asdf")
    
print("Success!")

## Further reading

The CRDS User Manual includes [detailed documentation](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html) on the subject of CRDS rules files.